In [2]:

from pyspark.sql import SparkSession



In [3]:
spark = (
    SparkSession
    .builder
    .appName("streaming with sockets")
    .master("local[*]")
    .getOrCreate()
)
spark

In [6]:
df_stream = spark.readStream.format("socket").option("host", "localhost").option("port", "9999").load()

In [7]:
df_stream.printSchema()

root
 |-- value: string (nullable = true)



In [19]:
from pyspark.sql.functions import split, explode, count, lit

df_words= df_stream.withColumn("words", split("value", " "))

In [22]:
df_exploded = df_words.withColumn("word", explode('words').alias("words")).drop("value", "words")

In [24]:
df_Agg = df_exploded.groupby("word").agg(count(lit(1)).alias("count"))

In [26]:
df_Agg.writeStream.format("console").outputMode("complete").start().awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 